### 1. Install dependencies

In [126]:
import os
from dotenv import load_dotenv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


### 2. FX scraper

In [129]:
url = "https://www.dbs.com/in/treasures/rates-online/foreign-currency-foreign-exchange.page"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

In [132]:
date_detail = driver.find_element(By.ID, "mainCurrency").text
elements = date_detail.split(' ')

date = elements[3]
time = elements[-1]

In [3]:
# Header Names
header = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/thead/tr/th")

col_names = list()

for head in header:
    col_names.append(head.text)

print(col_names)


['Currency', 'Selling TT', 'Selling Cash', 'Buying TT', 'Buying Cash']


In [4]:
# Locators
col = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr")
row = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr[1]/td")

num_row = len(row) #5
num_col = len(col) #10

results = []


for i in range (1, num_col + 1):
    result = []
    for j in range (1, num_row + 1):
        d = driver.find_element(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr["+str(i)+"]/td["+str(j)+"]").text
        result.append(d)
    results.append(result)

In [6]:
df = pd.DataFrame(data=results, columns=col_names)

df["Date"] = date
df["Time"] = time

print(df)

                      Currency Selling TT Selling Cash Buying TT Buying Cash  \
0                    US Dollar    81.2468      81.2468   78.0607     78.0607   
1             Singapore Dollar    59.4800      59.4800   56.8625     56.8625   
2            Australian Dollar    57.9094      57.9094   55.3610     55.3610   
3                  Swiss Franc    86.5179      86.5179   82.7106     82.7106   
4                         Euro    84.0371      84.0371   80.3389     80.3389   
5                British Pound    99.3098      99.3098   94.9394     94.9394   
6              Canadian Dollar    63.8483      63.8483   61.0385     61.0385   
7             Hong Kong Dollar    10.3947      10.3947    9.9373      9.9373   
8                 Japanese Yen     0.6111       0.6111    0.5842      0.5842   
9  United Arab Emirates Dirham    22.1744      22.1744   21.1986     21.1986   

         Date     Time  
0  12/08/2022  11:45AM  
1  12/08/2022  11:45AM  
2  12/08/2022  11:45AM  
3  12/08/2022  11:4

# 3. PDF File

In [37]:
pdf = "/home/oem/Documents/coding/personal/Reits-Analytics/data/Ascendas-Reit-Annual-Report-2021.pdf"

import tabula
import pandas as pd

tables = tabula.read_pdf(pdf, pages=[72])
df = tables[0]
df = df.rename(columns={"Unnamed: 0": "num", 
                "Unnamed: 1": "property", 
                "Business & Science Park Properties": "development_cost",	
                "Unnamed: 2": "valuations_(SGD:Mil)",
                "Unnamed: 3": "gfa_(Sqm)",
                "Unnamed: 4": "nla_(Sqm)",
                "Unnamed: 5": "address",
                "Unnamed: 6": "gross_revenue_(SGD:Mil)",
                "Unnamed: 7": "occupancy_rate"})
# remove first rows
df = df.iloc[4:]
# split out completion date parsed incorrectly
def split_completion_date(text):
    if isinstance(text, str):
        x = text[:9]
        return x

df["completion_date"] = df["development_cost"].apply(lambda x: split_completion_date(x))
# split out development cost parsed incorrectly
def split_development_cost(text):
    if isinstance(text, str) and len(text) > 11:
        amt = text[10:]
        return amt

df["development_cost"] = df["development_cost"].apply(lambda x: split_development_cost(x))
df = df.dropna(how="any", subset=["development_cost", "completion_date"])

charlist = ["#", "*", "^"]
for char in charlist:
    df["property"] = df["property"].str.replace(char, "", regex=True)


# 4. Geographical API

In [38]:
from dotenv import load_dotenv 
import os
import numpy as np
import requests

query = "dakota breeze"

class google_api:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv("GOOGLE_KEY")

    def return_lat_long(self, query, country):
        query = self.custom_query(query, country)
        path = f"https://maps.googleapis.com/maps/api/geocode/json?address={query}?,+CA&key={self.api_key}"
        response = requests.get(path)
        jsonResponse = response.json()
        if jsonResponse["status"]=="OK":
            lat = jsonResponse["results"][0]["geometry"]["location"]["lat"]
            long = jsonResponse["results"][0]["geometry"]["location"]["lng"]
            return lat, long
        return None

    def custom_query(self, query, country):
        if query == "1, 3 & 5 Changi Business Park Crescent":
            query = "3 Changi Business Park Crescent"
        query = query+f", {country}"
        query = query.replace(" ", "%20")

        return query

In [39]:
df["address_cord"] = df["address"].apply(lambda x: google_api().return_lat_long(x, "Singapore"))
df["property_cord"] = df["property"].apply(lambda x: google_api().return_lat_long(x, "Singapore"))

df["lat_lng"] = np.where(df["address_cord"].isna(),df["property_cord"],df["address_cord"])

df["lattitude"] = df["lat_lng"].apply(lambda x: x[0])
df["longitude"] = df["lat_lng"].apply(lambda x: x[1])

df.drop(columns=["address_cord", "property_cord", "lat_lng"], inplace=True)


In [40]:
floattypes = ["development_cost", "valuations_(SGD:Mil)", "gross_revenue_(SGD:Mil)"]

def to_numeric(number):
    try:
        return float(number)
    except:
        return 0

for each in floattypes:
    df[each] = df[each].apply(lambda x: to_numeric(x))

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 5 to 44
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num                      28 non-null     float64
 1   property                 28 non-null     object 
 2   development_cost         28 non-null     float64
 3   valuations_(SGD:Mil)     28 non-null     float64
 4   gfa_(Sqm)                28 non-null     object 
 5   nla_(Sqm)                28 non-null     object 
 6   address                  28 non-null     object 
 7   gross_revenue_(SGD:Mil)  28 non-null     float64
 8   occupancy_rate           28 non-null     object 
 9   completion_date          28 non-null     object 
 10  lattitude                28 non-null     float64
 11  longitude                28 non-null     float64
dtypes: float64(6), object(6)
memory usage: 2.8+ KB


# 5. Vizualization

In [59]:
import plotly.express as px
fig = px.scatter_mapbox(df, 
                        title="business park", 
                        lon="longitude", 
                        lat="lattitude", 
                        color="property", 
                        size="gross_revenue_(SGD:Mil)", 
                        zoom=4, 
                        width=900, 
                        height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(autosize=True, margin={"r": 10, "t": 50, "l": 10, "b": 10})
fig.update_layout(mapbox_bounds={"west": 0, "east": 1000, "south": 500, "north": 0})

fig.show()

In [276]:
df

,num,property,development_cost,valuations_(SGD:Mil),gfa_(Sqm),nla_(Sqm),address,gross_revenue_(SGD:Mil),occupancy_rate,completion_date,lat_lng
5,1.0,Neuros & Immunos,125.6,143.0,"36,931","26,035",8/8A Biomedical Grove,21.0,98.4%,31 Mar 11,"(1.3020901, 103.7927811)"
6,2.0,Nexus @one-north,181.3,202.8,"25,511","20,669",1 & 3 Fusionopolis Link,15.0,93.4%,04 Sep 13,"(1.2988017, 103.7899878)"
7,3.0,Nucleos,289.0,346.1,"46,174","37,365",21 Biopolis Road,30.6,98.4%,11 Dec 19,"(1.301487, 103.7921404)"
8,4.0,Galaxis,697.5,733.6,"68,835","60,709",1 & 3 Fusionopolis Place,25.7,97.0%,30 Jun 21,"(1.2998645, 103.7883994)"
9,5.0,Grab Headquarters,184.6,191.5,"42,290","42,290",1 & 3 Media Close,4.6,100.0%,30 Jul 21,"(1.2918178, 103.7930462)"
12,6.0,Techquest,7.5,24.9,"9,079","9,079",7 International Business Park,2.8,100.0%,05 Oct 05,"(1.3255407, 103.749825)"
13,7.0,iQuest@IBP,18.6,–,–,–,27 International Business Park,–,–,12 Jan 07,"(1.3258126, 103.7470642)"
14,8.0,Acer Building,75.0,79.1,"29,185","22,553",29 International Business Park,3.2,40.3%,19 Mar 08,"(1.327022, 103.7460233)"
15,9.0,31 International Business Park,246.8,206.1,"61,720","48,968",31 International Business Park,11.2,51.7%,26 Jun 08,"(1.3287102, 103.7461134)"
16,10.0,Nordic European Centre,121.6,121.1,"28,378","21,546",3 International Business Park,8.9,75.1%,08 Jul 11,"(1.3273051, 103.7485578)"


In [246]:
df.drop(columns=["address_cord","property_cord"])

,num,property,development_cost,valuations_(SGD:Mil),gfa_(Sqm),nla_(Sqm),address,gross_revenue_(SGD:Mil),occupancy_rate,completion_date,lat_lng
5,1.0,Neuros & Immunos,125.6,143.0,"36,931","26,035",8/8A Biomedical Grove,21.0,98.4%,31 Mar 11,"(1.3020901, 103.7927811)"
6,2.0,Nexus @one-north,181.3,202.8,"25,511","20,669",1 & 3 Fusionopolis Link,15.0,93.4%,04 Sep 13,"(1.2988017, 103.7899878)"
7,3.0,Nucleos,289.0,346.1,"46,174","37,365",21 Biopolis Road,30.6,98.4%,11 Dec 19,"(1.301487, 103.7921404)"
8,4.0,Galaxis,697.5,733.6,"68,835","60,709",1 & 3 Fusionopolis Place,25.7,97.0%,30 Jun 21,"(1.2998645, 103.7883994)"
9,5.0,Grab Headquarters,184.6,191.5,"42,290","42,290",1 & 3 Media Close,4.6,100.0%,30 Jul 21,"(1.2918178, 103.7930462)"
12,6.0,Techquest,7.5,24.9,"9,079","9,079",7 International Business Park,2.8,100.0%,05 Oct 05,"(1.3255407, 103.749825)"
13,7.0,iQuest@IBP,18.6,–,–,–,27 International Business Park,–,–,12 Jan 07,"(1.3258126, 103.7470642)"
14,8.0,Acer Building,75.0,79.1,"29,185","22,553",29 International Business Park,3.2,40.3%,19 Mar 08,"(1.327022, 103.7460233)"
15,9.0,31 International Business Park,246.8,206.1,"61,720","48,968",31 International Business Park,11.2,51.7%,26 Jun 08,"(1.3287102, 103.7461134)"
16,10.0,Nordic European Centre,121.6,121.1,"28,378","21,546",3 International Business Park,8.9,75.1%,08 Jul 11,"(1.3273051, 103.7485578)"
